In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark

your 131072x1 screen size is bogus. expect trouble
23/05/21 12:52:09 WARN Utils: Your hostname, DESKTOP-QQKQ81M resolves to a loopback address: 127.0.1.1; using 172.31.131.214 instead (on interface eth0)
23/05/21 12:52:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/21 12:52:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
# !wget https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
# !mv rows.csv\?accessType\=DOWNLOAD data/chicago_crime_data.csv

In [7]:
from pyspark.sql.functions import to_timestamp, col, lit
df = spark.read.csv('../data/chicago_crime_data.csv', header=True).withColumn('Date'), to_timestamp(col('Date'), 'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') <= lit('2018-11-11'))
df.show(5)

+--------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224738|   HY411648|2015-09-05 13:30:00|     043XX S WOOD ST|0486|     BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    true|0924|     00

In [8]:
df.dtypes

[('ID', 'string'),
 ('Case Number', 'string'),
 ('Date', 'timestamp'),
 ('Block', 'string'),
 ('IUCR', 'string'),
 ('Primary Type', 'string'),
 ('Description', 'string'),
 ('Location Description', 'string'),
 ('Arrest', 'string'),
 ('Domestic', 'string'),
 ('Beat', 'string'),
 ('District', 'string'),
 ('Ward', 'string'),
 ('Community Area', 'string'),
 ('FBI Code', 'string'),
 ('X Coordinate', 'string'),
 ('Y Coordinate', 'string'),
 ('Year', 'string'),
 ('Updated On', 'string'),
 ('Latitude', 'string'),
 ('Longitude', 'string'),
 ('Location', 'string')]

In [9]:
df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



Testing Schemas

In [30]:
from pyspark.sql.types import StructField, StructType, BooleanType, TimestampType, DoubleType, IntegerType, StringType

In [105]:
labels = [
    ('ID', IntegerType()),
    ('Case Number', StringType()),
    ('Date', TimestampType()),
    ('Block', StringType()),
    ('IUCR', IntegerType()),
    ('Primary Type', StringType()),
    ('Description', StringType()),
    ('Location Description', StringType()),
    ('Arrest', BooleanType()),
    ('Domestic', BooleanType()),
    ('Beat', IntegerType()),
    ('District', IntegerType()),
    ('Ward', IntegerType()),
    ('Community Area', IntegerType()),
    ('FBI Code', StringType()),
    ('X Coordinate', IntegerType()),
    ('Y Coordinate', IntegerType()),
    ('Year', IntegerType()),
    ('Updated On', TimestampType()),
    ('Latitude', DoubleType()),
    ('Longitude', DoubleType()),
    ('Location', StringType())]

colunas = []
tipos = []
for x in labels:
    colunas.append(x[0])
    tipos.append(x[1])


In [96]:
nulos = []
for x in df.columns:
    if df.filter(col(x).isNull()).count():
        nulos.append('true')
    else:
        nulos.append('false')


In [106]:
all_together = list(zip(colunas, tipos, nulos))

In [107]:
schema = StructType([StructField(x[0], x[1], True) for x in all_together])
schema

StructType([StructField('ID', IntegerType(), True), StructField('Case Number', StringType(), True), StructField('Date', TimestampType(), True), StructField('Block', StringType(), True), StructField('IUCR', IntegerType(), True), StructField('Primary Type', StringType(), True), StructField('Description', StringType(), True), StructField('Location Description', StringType(), True), StructField('Arrest', BooleanType(), True), StructField('Domestic', BooleanType(), True), StructField('Beat', IntegerType(), True), StructField('District', IntegerType(), True), StructField('Ward', IntegerType(), True), StructField('Community Area', IntegerType(), True), StructField('FBI Code', StringType(), True), StructField('X Coordinate', IntegerType(), True), StructField('Y Coordinate', IntegerType(), True), StructField('Year', IntegerType(), True), StructField('Updated On', TimestampType(), True), StructField('Latitude', DoubleType(), True), StructField('Longitude', DoubleType(), True), StructField('Locat

In [108]:
df = spark.read.csv('../data/chicago_crime_data.csv', schema=schema)
df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: integer (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- Community Area: integer (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: integer (nullable = true)
 |-- Y Coordinate: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: timestamp (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)



In [109]:
df.show(5)

+--------+-----------+----+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------+------------+-------------+--------------------+
|      ID|Case Number|Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+----+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------+------------+-------------+--------------------+
|    null|Case Number|null|               Block|null|Primary Type|         Description|Location Description|  null|    null|null|    null|null|          null|FBI Code|        null|        null|null|      null|        null|    

Working with Columns

In [110]:
df.Description

Column<'Description'>

In [111]:
df['Description']

Column<'Description'>

In [112]:
df.select(col('Description'))

DataFrame[Description: string]

In [118]:
df.select('Description', 'Location').show(5)

+--------------------+--------------------+
|         Description|            Location|
+--------------------+--------------------+
|         Description|            Location|
|DOMESTIC BATTERY ...|(41.815117282, -8...|
|      POCKET-PICKING|(41.895080471, -8...|
|           OVER $500|                null|
|POSS: HEROIN(BRN/...|(41.937405765, -8...|
+--------------------+--------------------+
only showing top 5 rows



In [144]:
#adding a column based on another column
df = df.withColumn('DoubleColumn', 2*df['Year'])
df.select('Year', 'DoubleColumn').show(5)


+----+------------+
|Year|DoubleColumn|
+----+------------+
|null|        null|
|2015|        4030|
|2015|        4030|
|2018|        4036|
|2015|        4030|
+----+------------+
only showing top 5 rows



In [145]:
#renaming a column
df = df.withColumnRenamed('DoubleColumn', 'YearTimesTwo')
df.select('YearTimesTwo').show(5)

+------------+
|YearTimesTwo|
+------------+
|        null|
|        4030|
|        4030|
|        4036|
|        4030|
+------------+
only showing top 5 rows



In [147]:
#dropping a column
df = df.drop('YearTimesTwo')

In [149]:
#group by
df.groupBy('YearTimesTwo').show(5)

AttributeError: 'GroupedData' object has no attribute 'show'

Working with rows

In [151]:
#filtering rows
df.filter(col('Year') > 2000).show(5)

+--------+-----------+----+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------+------------+-------------+--------------------+
|      ID|Case Number|Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+----+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------+------------+-------------+--------------------+
|10224738|   HY411648|null|     043XX S WOOD ST| 486|     BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    true| 924|       9|  12|            61|     08B|     1165074|     1875917|2015|      null|41.815117282|-87.

In [152]:
#unique values
df.select('Year').distinct().show()

+----+
|Year|
+----+
|2003|
|2007|
|2018|
|2015|
|2023|
|2006|
|2022|
|2013|
|null|
|2014|
|2019|
|2004|
|2020|
|2012|
|2009|
|2016|
|2001|
|2005|
|2010|
|2011|
+----+
only showing top 20 rows



In [154]:
#sorting rows
df.orderBy(col('Year')).select('Year').show(5)

+----+
|Year|
+----+
|null|
|2001|
|2001|
|2001|
|2001|
+----+
only showing top 5 rows



In [158]:
#unique and order by
df.select('Year').distinct().orderBy(col('Year')).select('Year').show()

+----+
|Year|
+----+
|null|
|2001|
|2002|
|2003|
|2004|
|2005|
|2006|
|2007|
|2008|
|2009|
|2010|
|2011|
|2012|
|2013|
|2014|
|2015|
|2016|
|2017|
|2018|
|2019|
+----+
only showing top 20 rows



In [170]:
#appending rows
df = spark.read.csv('../data/chicago_crime_data.csv', header=True).withColumn('Date', to_timestamp(col('Date'), 'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') < lit('2018-11-11'))
print(df.count())
df2 = spark.read.csv('../data/chicago_crime_data.csv', header=True).withColumn('Date', to_timestamp(col('Date'), 'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') == lit('2018-11-11'))
print(df2.count())

6756147


11


In [171]:
df = df.union(df2)
df.orderBy('Date', ascending=False).show(5)

+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|11504097|   JB511773|2018-11-11 00:00:00|003XX S ST LOUIS AVE|1320|   CRIMINAL DAMAGE|          TO VEHICLE|              STREET| fal

In [172]:
#top primary type of crimes
df.groupBy('Primary Type').count().orderBy('count', ascending=False).show()

+--------------------+-------+
|        Primary Type|  count|
+--------------------+-------+
|               THEFT|1418509|
|             BATTERY|1232278|
|     CRIMINAL DAMAGE| 771517|
|           NARCOTICS| 711770|
|       OTHER OFFENSE| 418903|
|             ASSAULT| 418521|
|            BURGLARY| 388041|
| MOTOR VEHICLE THEFT| 314133|
|  DECEPTIVE PRACTICE| 267073|
|             ROBBERY| 255602|
|   CRIMINAL TRESPASS| 193371|
|   WEAPONS VIOLATION|  70665|
|        PROSTITUTION|  68327|
|PUBLIC PEACE VIOL...|  47785|
|OFFENSE INVOLVING...|  46673|
| CRIM SEXUAL ASSAULT|  26476|
|         SEX OFFENSE|  25565|
|INTERFERENCE WITH...|  15140|
|            GAMBLING|  14422|
|LIQUOR LAW VIOLATION|  14068|
+--------------------+-------+
only showing top 20 rows



In [181]:
#what is the percentage of reported crimes that resulted in an arrest?
round((df.filter(col('Arrest')=='true').count() / df.count())*100,2)

27.75

In [182]:
#what are the top 3 locations for reported crimes?
df.groupBy('Location Description').count().orderBy('count', ascending=False).show(3)

+--------------------+-------+
|Location Description|  count|
+--------------------+-------+
|              STREET|1770624|
|           RESIDENCE|1146054|
|           APARTMENT| 699116|
+--------------------+-------+
only showing top 3 rows

